In [1]:
import pandas as pd
import sqlalchemy as sa


# Case BANK

## Conexão com SQL

In [15]:
def read_sql(query, engine):
    with engine.begin() as conn:
        results = pd.read_sql_query(sa.text(query), conn)
    return results
    
def execute_sql(query, engine):
    with engine.begin() as conn:
        results = conn.execute(
            sa.text(query)
            )
    return results

In [16]:
user = "root"
password = "swpmlu23-"
url_banco = "localhost"
nome_db = "bank"
conn_str = f"mysql+pymysql://{user}:{password}@{url_banco}/{nome_db}"
engine = sa.create_engine(conn_str)

## Criando Views

In [17]:
query_loan_client = '''
CREATE VIEW loan_client AS (
    SELECT 
        c.client_id,
        l.loan_id,
        STR_TO_DATE(a.`date`,"%y%m%d") AS dt_criacao_conta,
        STR_TO_DATE(l.`date` ,"%y%m%d") AS dt_emprestimo,
        CASE 
            WHEN l.status = "A" OR l.status = "C" THEN 0
            ELSE 1
        END AS status_devedor,
        l.amount,
        l.duration 
    FROM 
        bank.loan l JOIN
        bank.account a ON (l.account_id = a.account_id) JOIN 
        bank.disp d ON (a.account_id = d.account_id) JOIN 
        bank.client c ON (d.client_id = c.client_id)
    WHERE
        d.`type` = "OWNER"
    )
'''
execute_sql(query_loan_client, engine)

In [18]:
query_loan_trans = '''
CREATE VIEW loan_trans AS (
    SELECT
        l.loan_id,
        l.account_id,
        t.trans_id,
        STR_TO_DATE(l.`date`,"%y%m%d") AS dt_loan_date,
        STR_TO_DATE(t.`date`,"%y%m%d") AS dt_trans_date,
        CASE
            WHEN t.`type` = "PRIJEM" THEN "CREDITO"
            ELSE "RETIRADA"
        END AS tipo_trans,
        t.amount,
        CASE 
            WHEN t.`type` = "PRIJEM" THEN t.amount
            ELSE -1*t.amount
        END AS amount_com_sinal,
        t.balance,
        DATEDIFF(STR_TO_DATE(l.`date`,"%y%m%d"),  STR_TO_DATE(t.`date`,"%y%m%d")) AS dias_ate_emprestimo
    FROM 
        bank.loan l JOIN
        bank.trans t ON (l.account_id = t.account_id)
    HAVING 
        dt_trans_date <= dt_loan_date
    ORDER BY 
        l.loan_id,
        dt_trans_date
)
'''
execute_sql(query_loan_trans, engine)

## Análise de Transações

In [42]:
query_client_loan_trans = '''
    WITH agg_trans AS (
    SELECT
        lt.loan_id,
        SUM(
            CASE
                WHEN lt.tipo_trans = "CREDITO" THEN lt.amount
                ELSE 0
            END
        ) AS total_deposito,
        SUM(
            CASE
                WHEN lt.tipo_trans = "CREDITO" AND lt.dias_ate_emprestimo <= 90 THEN lt.amount
                ELSE 0
            END
        ) AS total_deposito_90,
        SUM(
            CASE
                WHEN lt.tipo_trans = "CREDITO" AND lt.dias_ate_emprestimo <= 90 THEN 1
                ELSE 0
            END
        ) AS num_deposito_90,
        SUM(
            CASE
                WHEN lt.tipo_trans = "RETIRADA" THEN lt.amount
                ELSE 0
            END
        ) AS total_retirada,
        SUM(
            CASE
                WHEN lt.tipo_trans = "RETIRADA" AND lt.dias_ate_emprestimo <= 90 THEN lt.amount
                ELSE 0
            END
        ) AS total_retirada_90,
        SUM(
            CASE
                WHEN lt.tipo_trans = "RETIRADA" AND lt.dias_ate_emprestimo <= 90 THEN 1
                ELSE 0
            END
        ) AS num_retirada_90,
        SUM(
            CASE
                WHEN lt.balance < 0 THEN 1
                ELSE 0
            END
        ) AS num_saldo_negativo,
        SUM(
            CASE
                WHEN lt.balance < 0 AND lt.dias_ate_emprestimo <= 90 THEN 1
                ELSE 0
            END
        ) AS num_saldo_negativo_90,
        AVG(lt.balance) AS saldo_medio
    FROM 
        bank.loan_trans lt 
    GROUP BY
        lt.loan_id)
SELECT 
    *
FROM 
    bank.loan_client lc JOIN
    agg_trans ON (lc.loan_id = agg_trans.loan_id)
'''
tb_clt = read_sql(query_client_loan_trans, engine)

In [43]:
tb_clt.columns

Index(['client_id', 'loan_id', 'dt_criacao_conta', 'dt_emprestimo',
       'status_devedor', 'amount', 'duration', 'loan_id', 'total_deposito',
       'total_deposito_90', 'num_deposito_90', 'total_retirada',
       'total_retirada_90', 'num_retirada_90', 'num_saldo_negativo',
       'num_saldo_negativo_90', 'saldo_medio'],
      dtype='object')

In [44]:
tb_clt.groupby('status_devedor')[['total_deposito',
       'total_deposito_90', 'num_deposito_90', 'total_retirada',
       'total_retirada_90', 'num_retirada_90', 'saldo_medio']].mean().reset_index().round(2)

,status_devedor,total_deposito,total_deposito_90,num_deposito_90,total_retirada,total_retirada_90,num_retirada_90,saldo_medio
0,0,297061.96,67257.48,6.34,281201.98,71197.78,11.24,44037.78
1,1,278733.17,57558.61,6.72,254299.29,61787.30,9.49,37093.79


In [45]:
tb_clt['prop_emprestimo_saldo'] = tb_clt['amount']/tb_clt['saldo_medio']
tb_clt.groupby('status_devedor')['prop_emprestimo_saldo'].mean()

status_devedor
0    3.496213
1    7.353995
Name: prop_emprestimo_saldo, dtype: float64

In [46]:
tb_clt['prop_emprestimo_deposito'] = tb_clt['amount']/tb_clt['total_deposito_90']
tb_clt.groupby('status_devedor')['prop_emprestimo_deposito'].mean()

status_devedor
0     15.239609
1    132.125712
Name: prop_emprestimo_deposito, dtype: float64

In [47]:
tb_clt.columns

Index(['client_id', 'loan_id', 'dt_criacao_conta', 'dt_emprestimo',
       'status_devedor', 'amount', 'duration', 'loan_id', 'total_deposito',
       'total_deposito_90', 'num_deposito_90', 'total_retirada',
       'total_retirada_90', 'num_retirada_90', 'num_saldo_negativo',
       'num_saldo_negativo_90', 'saldo_medio', 'prop_emprestimo_saldo',
       'prop_emprestimo_deposito'],
      dtype='object')

In [48]:
# Isto é uma gambiarra para não re-escrever nosso view!
tb_clt['parcela'] = tb_clt['amount']/tb_clt['duration']
tb_clt['deposito_medio_90'] = tb_clt['total_deposito_90']/tb_clt['num_deposito_90']
tb_clt['prop_parcela_deposito_mu'] = tb_clt['parcela']/tb_clt['deposito_medio_90']

In [49]:
tb_clt.groupby('status_devedor')['prop_parcela_deposito_mu'].mean()

status_devedor
0    1.382537
1    7.963385
Name: prop_parcela_deposito_mu, dtype: float64

In [51]:
tb_clt.groupby('status_devedor')[['num_saldo_negativo', 'num_saldo_negativo_90']].mean().reset_index()

,status_devedor,num_saldo_negativo,num_saldo_negativo_90
0,0,0.000000,0.000000
1,1,1.592105,0.907895


## Análise Demográfica

In [55]:
query_distrito = '''
WITH d AS
(
    SELECT
        d.A1 AS district_id,
        d.A2 AS district_name,
        d.A3 AS region_name,
        d.a4 AS population,
        d.A11 AS average_salary,
        d.A12 AS desemprego_95,
        d.A13 AS desemprego_96,
        d.A10 AS urbanizacao,
        d.a15/d.a4*1000 AS crimes_1k_95,
        d.a16/d.a4*1000 AS crimes_1k_96
    FROM 
        bank.district d
)
SELECT
    lc.*,
    d.*
FROM 
    d JOIN
    bank.client c ON (d.district_id = c.district_id) JOIN 
    bank.loan_client lc ON (c.client_id = lc.client_id)
'''
tb_distrito = read_sql(query_distrito, engine)

In [56]:
tb_distrito.head()

,client_id,loan_id,dt_criacao_conta,dt_emprestimo,status_devedor,amount,duration,district_id,district_name,region_name,population,average_salary,desemprego_95,desemprego_96,urbanizacao,crimes_1k_95,crimes_1k_96
0,2,4959,1993-02-26,1994-01-05,0,80952,24,1,Hl.m. Praha,Prague,1204953,12541,0.29,0.43,100.0,71.1040,82.2497
1,25,4961,1995-04-07,1996-04-29,1,30276,12,21,Tabor,south Bohemia,103347,9104,1.51,2.07,67.0,22.2454,22.7776
2,31,4962,1996-07-28,1997-12-08,0,30276,12,68,Frydek - Mistek,north Moravia,228848,9893,4.09,4.72,57.2,24.5709,25.7245
3,45,4967,1997-08-18,1998-10-14,1,318480,60,20,Strakonice,south Bohemia,70646,8547,2.65,3.64,58.4,22.1244,21.8271
4,46,4968,1997-08-08,1998-04-19,0,110736,48,19,Prachatice,south Bohemia,51428,8402,3.13,3.98,52.7,19.4252,21.3697


In [57]:
tb_distrito.columns

Index(['client_id', 'loan_id', 'dt_criacao_conta', 'dt_emprestimo',
       'status_devedor', 'amount', 'duration', 'district_id', 'district_name',
       'region_name', 'population', 'average_salary', 'desemprego_95',
       'desemprego_96', 'urbanizacao', 'crimes_1k_95', 'crimes_1k_96'],
      dtype='object')

In [61]:
tb_distrito.groupby('status_devedor')[['population', 'average_salary', 'desemprego_95', 'desemprego_96', 'urbanizacao', 'crimes_1k_95', 'crimes_1k_96']].mean()

,population,average_salary,desemprego_95,desemprego_96,urbanizacao,crimes_1k_95,crimes_1k_96
status_devedor,,,,,,,
0,267634.531353,9483.674917,2.858663,3.524191,67.897690,34.528956,36.703092
1,233625.881579,9354.052632,2.976974,3.770132,67.594737,33.108537,35.350088


In [64]:
tb_distrito['desemprego_95'].quantile([0.25, 0.5, 0.75])

0.25    1.51
0.50    2.77
0.75    3.85
Name: desemprego_95, dtype: float64